____
__Universidad de San Andrés__<br/>
__Visión Artificial__<br/>
__Trabajo Práctico 1: Panorámica__<br/>
__Martin Bianchi y Federico Gutman__
____

### Importamos las librerías necesarias

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

### Visualizamos las imágenes